In [69]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
%load_ext tensorboard

In [45]:
datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

'20221114-113321'

In [60]:
file_writer = tf.summary.SummaryWriter('/path/to/logs', sess.graph)


NameError: name 'sess' is not defined

In [3]:
!rm -rf ./logs/

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [46]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.2200 - accuracy: 0.9355 - val_loss: 0.1071 - val_accuracy: 0.9692
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0978 - accuracy: 0.9698 - val_loss: 0.0851 - val_accuracy: 0.9748
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0709 - accuracy: 0.9776 - val_loss: 0.0804 - val_accuracy: 0.9754
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0538 - accuracy: 0.9830 - val_loss: 0.0711 - val_accuracy: 0.9796
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0431 - accuracy: 0.9859 - val_loss: 0.0668 - val_accuracy: 0.9810


In [ ]:
'''When training with methods such as tf.GradientTape(), use tf.summary to log the required information.

Use the same dataset as above, but convert it to tf.data.Dataset to take advantage of batching capabilities:'''

In [50]:
train_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test))

train_dataset = train_dataset.shuffle(60000).batch(64)
test_dataset = test_dataset.batch(64)

In [52]:
loss_function=tf.keras.losses.SparseCategoricalCrossentropy()
optimizer=tf.keras.optimizers.Adam()

In [58]:
#Creating the loss function for each type of datasets.
#The names like train_loss, train_Acuracy are the names we have given to the values which will be stored
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [64]:
def train_step(model, optimizer, x_train,y_train):
    with tf.GradientTape() as tape:
        predictions=model(x_train,training=True)
        loss=loss_function(y_train, predictions)
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(y_train, predictions)
    
def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_function(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [62]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [65]:
model = create_model() # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
  for (x_train, y_train) in train_dataset:
    train_step(model, optimizer, x_train, y_train)
  with train_summary_writer.as_default():
    tf.summary.scalar('loss', train_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

  for (x_test, y_test) in test_dataset:
    test_step(model, x_test, y_test)
  with test_summary_writer.as_default():
    tf.summary.scalar('loss', test_loss.result(), step=epoch)
    tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

  # Reset metrics every epoch
  train_loss.reset_states()
  test_loss.reset_states()
  train_accuracy.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.22111776471138, Accuracy: 93.46833038330078, Test Loss: 0.09888975322246552, Test Accuracy: 97.08000183105469
Epoch 2, Loss: 0.08859297633171082, Accuracy: 97.31666564941406, Test Loss: 0.07095128297805786, Test Accuracy: 97.65999603271484
Epoch 3, Loss: 0.0620398074388504, Accuracy: 98.05999755859375, Test Loss: 0.06340519338846207, Test Accuracy: 97.93000030517578
Epoch 4, Loss: 0.04903946816921234, Accuracy: 98.48333740234375, Test Loss: 0.0603596456348896, Test Accuracy: 98.0999984741211
Epoch 5, Loss: 0.03732078894972801, Accuracy: 98.8316650390625, Test Loss: 0.06380463391542435, Test Accuracy: 98.02999877929688


In [71]:
'''C:/Users/HP/Applied AI/Assignment 20(Working with callbacks)/logs/gradient_tape'''
%tensorboard --logdir logs/gradient_tape


Reusing TensorBoard on port 6006 (pid 7344), started 0:12:27 ago. (Use '!kill 7344' to kill it.)